In [95]:
import numpy as np

In [451]:
def generate_TF_inner(arr, n):
        if n == 0:
            return arr
        else:
            new_arr = []
            for i in arr:
                for possible_value in (True, False):
                    new_i = i.copy()
                    new_i.append(possible_value)
                    new_arr.append(new_i)
            if (len(new_arr) == 0):
                new_arr = [[True],[False]]
            return generate_TF_inner(new_arr, n-1)
                    
# generate all possible values of True, False for n variables
def generate_TF(n):
    arr = []
    ans = generate_TF_inner(arr, n)
    return ans

In [20]:
def simplifyString(string):
    return string.replace(" ", "")
def fill_vars(string):
    vars = []
    for i in string:
        if (re.match('[a-zA-Z]', i)):
            vars.append(i)
    return vars
def fill_ops(string):
    ops = []
    for i in string:
        if (i in available_ops['parenthesis'] or \
            i in available_ops['binary'] or \
            i in available_ops['unary']):
            ops.append(i)
    return ops


Парсим выражение.
Есть операторы. Есть формулы. Формулы - это либо атомы, либо выражения. Объединение операторов и других формул даёт новую формулу.

Видим выражение (A&B)|(!A&(B|C))&D

1. Атом - инстанс класса, который хранит в себе номер аргумента из комбинации значений всех атомов в рамках одной интерпретации.
2. Делаем сет всех атомов.
3. Дробим исходное выражение на формулы. 
    1. Сначала отсекаем части по операторам (то есть что между скобок) - в рамках одного уровня заносим все встреченные операции. ['|', '&']
    2. Вместо скобок создаём формулы. [(A&B), (!A&(B|C), D]
    4. Эти штуки рекурсивно преобразуем по той же схеме. 
    5. После преобразования каждая штука должна возвратить формулу куда можно подставить комбинацию значений.
    6. Полученные формулы объединеям в одну с учетом стека операций и возвращаем.
4. Получится большая лямбда, которая принимает комбинацию всех значений.
5. Делаем таблицу.



In [543]:
class Atom:
    def __init__(self, name):
        self.name = name
    def substitute(self, args):
        return args[self.name]
    def __str__(self):
        return "Atom " + self.name
    def __repr__(self):
        return "Atom " + self.name

class Op:
    def __init__(self, op_string, unary):
        self.op_string = op_string
        self.unary = unary
    def __str__(self):
        return self.op_string
    def __repr__(self):
        return self.op_string
    def op_on(self, *lambdas):
        pass
    def identify(self, expr):
        if (expr.find(self.op_string) == 0):
            return True
        else:
            return False

class UnaryNeg(Op):
    def __init__(self):
        super().__init__("!", True)
    def op_on(self, *lambdas):
        return lambda x: not lambdas[0](x)
class AndOp(Op):
    def __init__(self):
        super().__init__("&", False)
    def op_on(self, *lambdas):
        return lambda x: lambdas[0](x) and lambdas[1](x)
class OrOp(Op):
    def __init__(self):
        super().__init__("|", False)
    def op_on(self, *lambdas):
        return lambda x: lambdas[0](x) or lambdas[1](x)
class ImplicatOp(Op):
    def __init__(self):
        super().__init__("->", False)
    def op_on(self, *lambdas):
        def implicat(x):
            a = lambdas[0](x)
            b = lambdas[1](x)
            if a and not b:
                return False
            else:
                return True
        return implicat
class EquivOp(Op):
    def __init__(self):
        super().__init__("~", False)
    def op_on(self, *lambdas):
        def equive(x):
            a = lambdas[0](x)
            b = lambdas[1](x)
            if (a and b) or (not a and not b):
                return True
            else:
                return False
        return equive
# try to identify the op in the given string and return it
def identify_op(expression):
    ops_set = {UnaryNeg(): True, AndOp(): True, OrOp(): True, ImplicatOp(): True, EquivOp(): True}
    for op in ops_set:
        if (op.identify(expression)):
            return op
    return None

In [544]:
a = UnaryNeg()
b = AndOp()
c = OrOp()
/print a,b
expr = "!&"
b.identify(expr)

! &


False

In [467]:
def create_atom_set(expression):
    atom_set = {}
    for i in expression:
        if i not in atom_set and re.match('[a-zA-Z]', i):
            atom_set[i] = Atom(i)
            print(i)
    return atom_set

def get_var_names(expression):
    names = []
    for i in expression:
        if i not in names and re.match('[a-zA-Z]', i):
            names.append(i)
    return names

In [354]:
def build_binary_lambda(op, lambda1, lambda2):
    if (op == '&'):
        def and_(x):
            return lambda1(x) and lambda2(x)
        return and_
    elif (op == '|'):
        def or_(x):
            return lambda1(x) or lambda2(x)
        return or_

In [431]:
def full_lambda_on_stacks(ops_stack, lambda_stack):
    lambda_stack_copy = lambda_stack.copy()
    current_lambda = []
    if (len(ops_stack) == 0 and len(lambda_stack_copy) == 1):
        return lambda_stack_copy[0]
    for i in range(0, len(ops_stack)):
        if (i == 0):
            current_lambda = build_binary_lambda(ops_stack[i] ,lambda_stack_copy[0], lambda_stack_copy[1])
            lambda_stack_copy = lambda_stack_copy[2:]
        else:
            current_lambda = build_binary_lambda(ops_stack[i], current_lambda, lambda_stack_copy[0])
            lambda_stack_copy = lambda_stack_copy[1:]
    return current_lambda

In [489]:
def lambda_for(expression, atom_set):
    # build variables dictionary
    if (atom_set == None):
        expression = simplifyString(expression)
        atom_set = create_atom_set(expression)
    ops_stack = []
    parenth_stack = []
    lambdas_stack = []
    available_ops = ['&', '|', '!']
    begin = 0
    for char_index in zip(expression, range(0, len(expression))):
        i = char_index[1]
        char = char_index[0]
        if (char == '(' and len(parenth_stack) == 0):
            parenth_stack.append('(')
            begin = i
        elif (char == '('):
            parenth_stack.append('(')
        elif (char == ')' and len(parenth_stack) == 1):
            parenth_stack.pop()
            inner_expr = expression[begin+1:i]
            # parse inner expression and build lambda
            inner_lambda = lambda_for(inner_expr, atom_set)
            print(inner_lambda)
            last_op = ops_stack[-1]
            if (last_op == '!'):
                lambdas_stack.append(lambda x: not inner_lambda(x))
                ops_stack.pop()
            elif (last_op == '&'):
                lambdas_stack.append(inner_lambda)
            elif (last_op == '|'):
                lambdas_stack.append(inner_lambda)
            begin = i+1
        elif (char == ')'):
            parenth_stack.pop()
        elif (char in available_ops and len(parenth_stack) == 0):
            print('added op', char)
            ops_stack.append(char)
        elif (re.match('[a-zA-Z]', char) and len(parenth_stack) == 0):
            print('match')
            if (len(ops_stack) == 0):
                print('added var', char)
                lambdas_stack.append(atom_set[char].substitute)
            else:
                last_op = ops_stack[-1]
                if (last_op == '!'):
                    char_loc = char
                    print('neg op on char', char_loc)
                    lambdas_stack.append(lambda x: not atom_set[char_loc].substitute(x))
                    ops_stack.pop()
                elif (last_op == '&'):
                    print('and op on char', char)
                    lambdas_stack.append(atom_set[char].substitute)
                elif (last_op == '|'):
                    print('or op on char', char)
                    lambdas_stack.append(atom_set[char].substitute)
    # after loop collect the stacks
    full_lambda = full_lambda_on_stacks(ops_stack, lambdas_stack)
    return full_lambda

In [474]:

def true_table_for(fun, var_names):
    table = []
    result_keyword = "result"
    all_combs = generate_TF(len(var_names))
    for i in all_combs:
        new_comb = {}
        for char_with_index in zip(var_names, range(0, len(var_names))):
            index = char_with_index[1]
            char = char_with_index[0]
            new_comb[char] = i[index]
        result = fun(new_comb)
        new_comb[result_keyword] = result
        table.append(new_comb)
    return table
def full_solve(expr):
    fun = lambda_for(expr, None)
    return true_table_for(fun, get_var_names(expr))

In [475]:
expression = '!A&B|(!A&C|(A&B))'

In [476]:
l = lambda_for(expression, None)
/print l

A
B
C
added op !
match
neg op on char A
added op &
match
and op on char B
added op |
added op !
match
neg op on char A
added op &
match
and op on char C
added op |
match
added var A
added op &
match
and op on char B
<function build_binary_lambda.<locals>.and_ at 0x7f9196f5b700>
<function build_binary_lambda.<locals>.or_ at 0x7f9196f5b940>
<function build_binary_lambda.<locals>.or_ at 0x7f9196f5bb80>


In [477]:
l({'A': False, 'B': False, 'C': True})

True

In [478]:
true_table_for(l, ['A', 'B', 'C'])

[{'A': True, 'B': True, 'C': True, 'result': True},
 {'A': True, 'B': True, 'C': False, 'result': True},
 {'A': True, 'B': False, 'C': True, 'result': False},
 {'A': True, 'B': False, 'C': False, 'result': False},
 {'A': False, 'B': True, 'C': True, 'result': True},
 {'A': False, 'B': True, 'C': False, 'result': True},
 {'A': False, 'B': False, 'C': True, 'result': True},
 {'A': False, 'B': False, 'C': False, 'result': False}]

In [479]:
expr2 = '!(A&B)'
l_2 = lambda_for(expr2, None)
true_table_for(l_2, ['A','B'])

A
B
added op !
match
added var A
added op &
match
and op on char B
<function build_binary_lambda.<locals>.and_ at 0x7f919aaa2a60>


[{'A': True, 'B': True, 'result': False},
 {'A': True, 'B': False, 'result': True},
 {'A': False, 'B': True, 'result': True},
 {'A': False, 'B': False, 'result': True}]

In [482]:
full_solve("A & !B")

A
B
match
added var A
added op &
added op !
match
neg op on char B


[{'A': True, 'B': True, 'result': False},
 {'A': True, 'B': False, 'result': True},
 {'A': False, 'B': True, 'result': False},
 {'A': False, 'B': False, 'result': False}]

In [488]:
full_solve("A & !B | !(C&D&!(C&E))")

A
B
C
D
E
match
added var A
added op &
added op !
match
neg op on char B
added op |
added op !
match
added var C
added op &
match
and op on char D
added op &
added op !
match
added var C
added op &
match
and op on char E
<function build_binary_lambda.<locals>.and_ at 0x7f9196f5b550>
<function build_binary_lambda.<locals>.and_ at 0x7f9196f5ba60>


[{'A': True, 'B': True, 'C': True, 'D': True, 'E': True, 'result': True},
 {'A': True, 'B': True, 'C': True, 'D': True, 'E': False, 'result': False},
 {'A': True, 'B': True, 'C': True, 'D': False, 'E': True, 'result': True},
 {'A': True, 'B': True, 'C': True, 'D': False, 'E': False, 'result': True},
 {'A': True, 'B': True, 'C': False, 'D': True, 'E': True, 'result': True},
 {'A': True, 'B': True, 'C': False, 'D': True, 'E': False, 'result': True},
 {'A': True, 'B': True, 'C': False, 'D': False, 'E': True, 'result': True},
 {'A': True, 'B': True, 'C': False, 'D': False, 'E': False, 'result': True},
 {'A': True, 'B': False, 'C': True, 'D': True, 'E': True, 'result': True},
 {'A': True, 'B': False, 'C': True, 'D': True, 'E': False, 'result': True},
 {'A': True, 'B': False, 'C': True, 'D': False, 'E': True, 'result': True},
 {'A': True, 'B': False, 'C': True, 'D': False, 'E': False, 'result': True},
 {'A': True, 'B': False, 'C': False, 'D': True, 'E': True, 'result': True},
 {'A': True, 'B

In [584]:
def collect_as_labmda(ops_stack, lambda_stack):
    lambda_stack_copy = lambda_stack.copy()
    current_lambda = None
    if (len(lambda_stack_copy) == 1):
        return lambda_stack_copy[0]
    for i in range(0, len(ops_stack)):
        if (current_lambda == None):
            current_lambda = ops_stack[i].op_on(lambda_stack_copy[0], lambda_stack_copy[1])
            lambda_stack_copy = lambda_stack_copy[2:]
        else:
            current_lambda = ops_stack[i].op_on(current_lambda, lambda_stack_copy[0])
            lambda_stack_copy = lambda_stack_copy[1:]
    return current_lambda

def lambda_for_adv(expression, atom_set):
    # build variables dictionary
    if (atom_set == None):
        expression = simplifyString(expression)
        atom_set = create_atom_set(expression)
    ops_stack = []
    parenth_stack = []
    lambdas_stack = []        
    begin = 0
    for char_index in zip(expression, range(0, len(expression))):
        i = char_index[1]
        char = char_index[0]
        if (char == '(' and len(parenth_stack) == 0):
            parenth_stack.append('(')
            begin = i
        elif (char == ')' and len(parenth_stack) == 1):
            parenth_stack.pop()
            inner_expr = expression[begin+1:i]
            # parse inner expression and build lambda
            inner_lambda = lambda_for_adv(inner_expr, atom_set)
            print(inner_lambda)
            last_op = ops_stack[-1]
            if (last_op.unary):
                lambdas_stack.append(last_op.op_on(inner_lambda))
                ops_stack.pop()
            else:
                lambdas_stack.append(inner_lambda)
            begin = i+1
        elif (char == '('):
            parenth_stack.append('(')
        elif (char == ')'):
            parenth_stack.pop()
        elif (len(parenth_stack) == 0):
            # now need to understand what kind of sumbol encountered
            identified_op = identify_op(expression[i:])
            if (identified_op != None):
                print(f"identified op: {identified_op}")
                ops_stack.append(identified_op)
            elif re.match('[a-zA-Z]', char):
                # if a char than just collect as lambda
                if (len(ops_stack) == 0):
                    print('added var', char)
                    lambdas_stack.append(atom_set[char].substitute)
                else:
                    # if operation check if can collect with the new char
                    last_op = ops_stack[-1]
                    if (last_op.unary):
                        lambdas_stack.append(last_op.op_on(atom_set[char].substitute))
                        ops_stack.pop()
                    else:
                        lambdas_stack.append(atom_set[char].substitute)
    # now the ops and lambdas must be filled
    full_lambda = collect_as_labmda(ops_stack, lambdas_stack)
    return full_lambda

In [585]:
la = lambda_for_adv("!(A)&(B->C)", None)

A
B
C
identified op: !
added var A
<bound method Atom.substitute of Atom A>
identified op: &
added var B
identified op: ->
<function ImplicatOp.op_on.<locals>.implicat at 0x7f919b0f8550>


In [586]:
la({'A': False, 'B': True, 'C': False})

False